In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python

In [2]:
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
from fedml import Alliance, AllianceMember,KerasSequentialBaseLearner, FedAveragingClassifier
from read_data import read_training_data
import numpy as np

alliance = Alliance()
M=10
def split_data(x,y,fractions):
    s = len(x)
    new_order = np.random.permutation(s)
    x = x[new_order]
    y = y[new_order]
    if type(fractions) is int:
        fraction_splits = np.int32(np.linspace(0,s,fractions+1))
        nr = fractions
    elif type(fractions)==np.ndarray or type(fractions)==list:
        parts = np.array(fractions)*s/np.sum(fractions)
        fraction_splits = np.int32([np.sum(parts[:i]) for i in range(len(parts)+1)])
        nr = len(parts)
        
    x_list = []
    y_list = []
    for i in range(nr):
        x_list.append(x[fraction_splits[i]:fraction_splits[i+1]]) 
        y_list.append(y[fraction_splits[i]:fraction_splits[i+1]])

    return x_list,y_list
F = [10,10,8,8,6,6,4,4,2,2,1,1]
M = len(F)
x_train_splits, y_train_splits = split_data(x_train,y_train,fractions=F)
x_test_splits, y_test_splits = split_data(x_test,y_test,fractions=F)
print("x_train shape: ", np.array(x_train).shape)
tot_len = 0
for split in x_train_splits:
    print("x_train_splits shape: ", split.shape)
    tot_len += len(split)
print("tot len: ", tot_len)
alliance.set_validation_dataset(x_test,y_test)

classes = np.arange(10)
for i in range(M):
    member = AllianceMember(x_train_splits[i], y_train_splits[i], classes=classes)
    alliance.add_member(member)

x_train shape:  (50000, 32, 32, 3)
x_train_splits shape:  (8064, 32, 32, 3)
x_train_splits shape:  (8065, 32, 32, 3)
x_train_splits shape:  (6451, 32, 32, 3)
x_train_splits shape:  (6452, 32, 32, 3)
x_train_splits shape:  (4838, 32, 32, 3)
x_train_splits shape:  (4839, 32, 32, 3)
x_train_splits shape:  (3226, 32, 32, 3)
x_train_splits shape:  (3226, 32, 32, 3)
x_train_splits shape:  (1613, 32, 32, 3)
x_train_splits shape:  (1613, 32, 32, 3)
x_train_splits shape:  (806, 32, 32, 3)
x_train_splits shape:  (807, 32, 32, 3)
tot len:  50000


In [4]:
from cifarmodel import KerasSequentialCifar

base_learner = KerasSequentialCifar()
fedavg_model = FedAveragingClassifier(alliance=alliance,base_learner=base_learner)


In [5]:
parameters = {"nr_global_iterations": 100, "nr_local_iterations":1, "training_steps": None, "data_augmentation": True, "model_size_averaging": True} 
training_loss, test_loss = fedavg_model.fit(parameters)

fit starts!
alliance temp model:  None
alliance temp model:  <cifarmodel.KerasSequentialCifar object at 0x7fba849274e0>
global epoch:  0
virtual memory used:  14.8 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 19s 75ms/step - loss: 2.3407 - accuracy: 0.1873
training starts model size:  6452
training steps:  None
Epoch 1/1
202/202 [==============================] - 14s 67ms/step - loss: 2.4628 - accuracy: 0.1589
training starts model size:  806
training steps:  None
Epoch 1/1
26/26 [==============================] - 3s 103ms/step - loss: 2.7674 - accuracy: 0.1241
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 81ms/step - loss: 2.6999 - accuracy: 0.1469
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 82ms/step - loss: 2.7194 - accuracy: 0.1228
training starts model size:  8065
training steps:  None
Ep

101/101 [==============================] - 7s 72ms/step - loss: 1.7482 - accuracy: 0.3469
training time:  103.7263 s.
average model starts
average model ends:  0.4703 s.
alliance_test_loss
alliance_test_loss
test_loss:  [0.102 0.213 0.367 0.453]
global epoch:  4
virtual memory used:  47.4 %
training starts model size:  8065
training steps:  None
Epoch 1/1
253/253 [==============================] - 15s 60ms/step - loss: 1.6541 - accuracy: 0.3986
training starts model size:  807
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 60ms/step - loss: 1.6559 - accuracy: 0.3965
training starts model size:  806
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 65ms/step - loss: 1.6358 - accuracy: 0.3945
training starts model size:  6452
training steps:  None
Epoch 1/1
202/202 [==============================] - 13s 64ms/step - loss: 1.6679 - accuracy: 0.3985
training starts model size:  6451
training steps:  None
Epoch 1/1
202/202 [=============

51/51 [==============================] - 3s 65ms/step - loss: 1.4264 - accuracy: 0.4960
training starts model size:  807
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 75ms/step - loss: 1.4601 - accuracy: 0.4808
training time:  103.5659 s.
average model starts
average model ends:  0.6414 s.
alliance_test_loss
alliance_test_loss
test_loss:  [0.102 0.213 0.367 0.453 0.46  0.513 0.499 0.542]
global epoch:  8
virtual memory used:  45.5 %
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 73ms/step - loss: 1.4774 - accuracy: 0.4730
training starts model size:  4838
training steps:  None
Epoch 1/1
152/152 [==============================] - 10s 67ms/step - loss: 1.4587 - accuracy: 0.4754
training starts model size:  4839
training steps:  None
Epoch 1/1
152/152 [==============================] - 9s 62ms/step - loss: 1.4648 - accuracy: 0.4784
training starts model size:  6451
training steps:  None
Epoch 1/1


51/51 [==============================] - 4s 69ms/step - loss: 1.3519 - accuracy: 0.5139
training starts model size:  8065
training steps:  None
Epoch 1/1
253/253 [==============================] - 16s 63ms/step - loss: 1.3491 - accuracy: 0.5184
training starts model size:  6452
training steps:  None
Epoch 1/1
202/202 [==============================] - 11s 56ms/step - loss: 1.3559 - accuracy: 0.5138
training time:  101.7585 s.
average model starts
average model ends:  0.3667 s.
alliance_test_loss
alliance_test_loss
test_loss:  [0.102 0.213 0.367 0.453 0.46  0.513 0.499 0.542 0.516 0.544 0.587 0.577]
global epoch:  12
virtual memory used:  50.5 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 17s 69ms/step - loss: 1.3270 - accuracy: 0.5273
training starts model size:  806
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 81ms/step - loss: 1.3185 - accuracy: 0.5236
training starts model size:  807
trai

26/26 [==============================] - 2s 58ms/step - loss: 1.2190 - accuracy: 0.5943
training starts model size:  4839
training steps:  None
Epoch 1/1
152/152 [==============================] - 9s 60ms/step - loss: 1.2654 - accuracy: 0.5623
training starts model size:  6452
training steps:  None
Epoch 1/1
202/202 [==============================] - 12s 62ms/step - loss: 1.2712 - accuracy: 0.5626
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 3s 63ms/step - loss: 1.2021 - accuracy: 0.5921
training starts model size:  4838
training steps:  None
Epoch 1/1
152/152 [==============================] - 10s 67ms/step - loss: 1.2604 - accuracy: 0.5556
training time:  100.8278 s.
average model starts
average model ends:  0.3601 s.
alliance_test_loss
alliance_test_loss
test_loss:  [0.102 0.213 0.367 0.453 0.46  0.513 0.499 0.542 0.516 0.544 0.587 0.577
 0.585 0.611 0.624 0.642]
global epoch:  16
virtual memory used:  48.0 %
training sta

...

152/152 [==============================] - 10s 69ms/step - loss: 0.7729 - accuracy: 0.7419
training starts model size:  6451
training steps:  None
Epoch 1/1
202/202 [==============================] - 13s 67ms/step - loss: 0.7747 - accuracy: 0.7341
training starts model size:  6452
training steps:  None
Epoch 1/1
202/202 [==============================] - 13s 65ms/step - loss: 0.7640 - accuracy: 0.7388
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 71ms/step - loss: 0.7498 - accuracy: 0.7539
training starts model size:  3226
training steps:  None
Epoch 1/1
101/101 [==============================] - 7s 70ms/step - loss: 0.7794 - accuracy: 0.7319
training time:  102.3629 s.
average model starts
average model ends:  0.406 s.
alliance_test_loss
alliance_test_loss
test_loss:  [0.102 0.213 0.367 0.453 0.46  0.513 0.499 0.542 0.516 0.544 0.587 0.577
 0.585 0.611 0.624 0.642 0.636 0.655 0.661 0.671 0.683 0.687 0.687 0.689
 0.694 0.6

26/26 [==============================] - 2s 81ms/step - loss: 0.8419 - accuracy: 0.7385
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 76ms/step - loss: 0.7306 - accuracy: 0.7619
training time:  104.3372 s.
average model starts
average model ends:  0.6575 s.
alliance_test_loss
alliance_test_loss
test_loss:  [0.102 0.213 0.367 0.453 0.46  0.513 0.499 0.542 0.516 0.544 0.587 0.577
 0.585 0.611 0.624 0.642 0.636 0.655 0.661 0.671 0.683 0.687 0.687 0.689
 0.694 0.697 0.711 0.707 0.719 0.718 0.721 0.742 0.724 0.724 0.728 0.739
 0.737 0.747 0.738 0.747 0.743 0.751 0.753 0.755 0.756 0.757 0.758 0.758
 0.763 0.759 0.769 0.766 0.758 0.77  0.776 0.778 0.78  0.775 0.768 0.775
 0.775 0.775 0.778 0.777 0.78  0.782 0.781 0.787 0.783 0.789 0.793 0.786
 0.79  0.779 0.795 0.797 0.791 0.802 0.793 0.789 0.797 0.796 0.802 0.794
 0.796 0.798 0.802 0.796 0.799]
global epoch:  89
virtual memory used:  53.2 %
training starts model size:  8064
trai

26/26 [==============================] - 2s 82ms/step - loss: 0.7449 - accuracy: 0.7534
training starts model size:  6451
training steps:  None
Epoch 1/1
202/202 [==============================] - 12s 60ms/step - loss: 0.7578 - accuracy: 0.7433
training starts model size:  4839
training steps:  None
Epoch 1/1
152/152 [==============================] - 10s 66ms/step - loss: 0.7524 - accuracy: 0.7425
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 3s 68ms/step - loss: 0.7430 - accuracy: 0.7477
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 69ms/step - loss: 0.7601 - accuracy: 0.7322
training starts model size:  8065
training steps:  None
Epoch 1/1
253/253 [==============================] - 15s 59ms/step - loss: 0.7178 - accuracy: 0.7566
training starts model size:  3226
training steps:  None
Epoch 1/1
101/101 [==============================] - 7s 65ms/step - loss: 0.7

152/152 [==============================] - 10s 65ms/step - loss: 0.7487 - accuracy: 0.7466
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 16s 64ms/step - loss: 0.7256 - accuracy: 0.7493
training starts model size:  806
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 69ms/step - loss: 0.7663 - accuracy: 0.7407
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 3s 65ms/step - loss: 0.6968 - accuracy: 0.7669
training starts model size:  3226
training steps:  None
Epoch 1/1
101/101 [==============================] - 7s 66ms/step - loss: 0.7668 - accuracy: 0.7492
training starts model size:  6452
training steps:  None
Epoch 1/1
202/202 [==============================] - 13s 63ms/step - loss: 0.7346 - accuracy: 0.7472
training starts model size:  8065
training steps:  None
Epoch 1/1
253/253 [==============================] - 16s 63ms/step - loss: 0

Epoch 1/1
152/152 [==============================] - 9s 61ms/step - loss: 0.7456 - accuracy: 0.7526
training starts model size:  4838
training steps:  None
Epoch 1/1
152/152 [==============================] - 9s 58ms/step - loss: 0.7583 - accuracy: 0.7420
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 15s 60ms/step - loss: 0.7434 - accuracy: 0.7464
training starts model size:  1613
training steps:  None
Epoch 1/1
51/51 [==============================] - 4s 72ms/step - loss: 0.6998 - accuracy: 0.7663
training starts model size:  807
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 69ms/step - loss: 0.6875 - accuracy: 0.7658
training starts model size:  806
training steps:  None
Epoch 1/1
26/26 [==============================] - 2s 71ms/step - loss: 0.7388 - accuracy: 0.7754
training starts model size:  8065
training steps:  None
Epoch 1/1
253/253 [==============================] - 16s 62ms/step - lo